In [0]:
from pyspark.sql.functions import col, radians, sin, cos, acos, lit, datediff, hour, dayofweek, avg
from pyspark.sql.window import Window

# --- CONFIGURATION ---
CATALOG = "project_fraud_detection"
SILVER_TABLE = f"{CATALOG}.silver_layer.cleaned_transactions"
GOLD_TABLE = f"{CATALOG}.gold_layer.model_features"

def calculate_distance_km(lat1, long1, lat2, long2):
    lat1, long1 = radians(lat1), radians(long1)
    lat2, long2 = radians(lat2), radians(long2)
    return (lit(6371.0) * acos(
        (sin(lat1) * sin(lat2)) + 
        (cos(lat1) * cos(lat2) * cos(long2 - long1))
    )).alias("distance_km")

def create_gold_features():
    print(f"Reading from Silver: {SILVER_TABLE}")
    df_silver = spark.table(SILVER_TABLE)

    # --- 1. DEFINE WINDOW FOR ADVANCED CONTEXT ---
    # This groups data by 'category' to calculate averages dynamically
    cat_window = Window.partitionBy("category")

    # --- 2. FEATURE ENGINEERING ---
    df_features = df_silver.select(
        col("is_fraud"),
        col("amount"),
        col("category"),
        col("transaction_time"),
        
        # TIME FEATURES
        hour("transaction_time").alias("hour_of_day"),
        dayofweek("transaction_time").alias("day_of_week"), # <--- NEW: 1=Sun, 7=Sat
        
        # DEMOGRAPHICS
        col("dob"),
        (datediff(col("transaction_time"), col("dob")) / 365).cast("int").alias("customer_age"),
        col("gender"),
        
        # LOCATION
        calculate_distance_km(
            col("cust_lat"), col("cust_long"),
            col("merch_lat"), col("merch_long")
        ),
        
        col("ingestion_time")
    )

    # --- 3. ADDING "SMART" CONTEXTUAL FEATURE ---
    # Calculate: How much bigger is this transaction than the category average?
    df_smart = df_features.withColumn(
        "category_avg_amt", avg("amount").over(cat_window)
    ).withColumn(
        "amt_relative_to_category", col("amount") / col("category_avg_amt")
    ).drop("category_avg_amt") # Drop the helper column, we just need the ratio

    # --- 4. CLEANUP & WRITE ---
    df_final = df_smart.na.fill(0, subset=["distance_km"])

    print(f"Writing features to Gold: {GOLD_TABLE}")
    (df_final.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(GOLD_TABLE))
    
    print(f"Success! Added 'day_of_week' and 'amt_relative_to_category'.")
    display(df_final.select("category", "amount", "amt_relative_to_category", "is_fraud").orderBy(col("amt_relative_to_category").desc()).limit(10))

# --- EXECUTE ---
create_gold_features()

Reading from Silver: project_fraud_detection.silver_layer.cleaned_transactions
Writing features to Gold: project_fraud_detection.gold_layer.model_features
Success! Added 'day_of_week' and 'amt_relative_to_category'.


category,amount,amt_relative_to_category,is_fraud
travel,192.245,6.1533244175495065,0
travel,192.245,6.1533244175495065,0
travel,192.245,6.1533244175495065,0
travel,192.245,6.1533244175495065,0
travel,192.245,6.1533244175495065,0
travel,192.245,6.1533244175495065,0
travel,192.245,6.1533244175495065,0
travel,192.245,6.1533244175495065,0
travel,192.245,6.1533244175495065,0
travel,192.245,6.1533244175495065,0


In [0]:
%sql
SELECT 
  is_fraud,
  ROUND(AVG(distance_km), 2) as avg_distance,
  ROUND(AVG(amt_relative_to_category), 2) as avg_relative_amt
FROM project_fraud_detection.gold_layer.model_features
GROUP BY is_fraud

is_fraud,avg_distance,avg_relative_amt
0,76.11,0.99
1,76.27,2.65
